In [1]:
import tensorflow as tf
import numpy as np
import time
import os

In [2]:
from keras.datasets import mnist, cifar10
(img_train, label_train), (img_test, label_test) = mnist.load_data()

C:\Users\Hyoje\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def Conv_block(inputs, filters, scope_num):
    with tf.variable_scope('{}/conv_layer'.format(model, scope_num)):
        conv = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=[3, 3], 
                                padding='SAME', activation=tf.nn.relu, 
                                name='conv{}'.format(scope_num))
        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], 
                                       padding='SAME', strides=2, 
                                       name='pool{}'.format(scope_num))
    return pool

In [4]:
def get_accuracy(sess, logits, img, label, batch_size):    
    if len(img) < batch_size:
        logit = sess.run(logits, feed_dict={X: img, training: False})
    else:
        total_batch = np.ceil((len(img) / batch_size)).astype(np.int32)
        num_classes = int(logits.shape[-1])
        logit = np.array([], dtype=np.int64).reshape(0, num_classes)
        idxs = range(len(img))
        for i in range(total_batch):
            idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
            feed_dict = {X: img[idxs_i], training: False}
            logit = np.concatenate((logit, sess.run(logits, feed_dict=feed_dict)), axis=0)

    correct_pred = np.equal(np.argmax(logit, 1), label).astype(np.float32)
    accuracy = np.mean(correct_pred)
    
    return accuracy, logit, correct_pred

In [5]:
model = 'basic'
img_row, img_col = img_train.shape[1], img_train.shape[2]
img_depth = 1

img_train = img_train.reshape(img_train.shape[0], img_row, img_col, img_depth)
img_test = img_test.reshape(img_test.shape[0], img_row, img_col, img_depth)
img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

In [6]:
X = tf.placeholder(tf.float32, [None, img_row, img_col, img_depth])
Y = tf.placeholder(tf.int64, [None])
training = tf.placeholder(tf.bool)

In [7]:
# build network
# convolution layers

block1 = Conv_block(inputs=X, filters=32, scope_num=1)
block2 = Conv_block(inputs=block1, filters=32, scope_num=2)
conv = Conv_block(inputs=block2, filters=32, scope_num=3)

In [8]:
# fully connected layers

flat_num = int(conv.shape[1] * conv.shape[2] * conv.shape[3])
flat = tf.reshape(conv, [-1, flat_num])

with tf.variable_scope('{}/fc_layer'.format(model)):
    dense1 = tf.layers.dense(inputs=flat, units=512, 
                            activation=tf.nn.relu, name='dense1')
    drop1 = tf.layers.dropout(inputs=dense1, rate=0.25, training=training)
    dense2 = tf.layers.dense(inputs=drop1, units=512, 
                            activation=tf.nn.relu, name='dense2')
    drop2 = tf.layers.dropout(inputs=dense2, rate=0.25, training=training)
    logits = tf.layers.dense(inputs=drop2, units=10, name='logit')        

In [9]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# hyper prameter
BATCH_SIZE = 128
EPOCHS = 5
CHECK_POINT_DIR = './save_model'
IS_SAVE = False

In [11]:
config=tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allocator_type='BFC'
config.log_device_placement=False
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [12]:
# Restore and Save
if IS_SAVE:
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state(CHECK_POINT_DIR)

    if checkpoint and checkpoint.model_checkpoint_path:
        try:
            saver.restore(sess, checkpoint.model_checkpoint_path)
            print("Successfully loaded:", checkpoint.model_checkpoint_path)
        except:
            print("Error on loading old network weights")
    else:
        print("Could not find old network weights")
else:
    print('Do not save')

Do not save


In [15]:
# train

print(time.asctime())
print('Batch size : {}, Epochs : {}, Save_dir : {}'.format(BATCH_SIZE, EPOCHS, CHECK_POINT_DIR))
print('Learning Started.')

total_start = time.time()
max_accuracy = 0.0
for epoch in range(EPOCHS):
    epoch_start = time.time()
    avg_cost = 0
    total_batch = np.ceil(img_train.shape[0] / BATCH_SIZE).astype(np.int32)
    
    idxs = np.random.permutation(range(len(img_train)))
    for i in range(total_batch):
        idxs_i = idxs[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i], training: True }    
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('=====================================')
    print('Epoch : {:02d} , Cost : {:.5f}'.format(epoch + 1, avg_cost))
    train_accuracy, _, _ = get_accuracy(sess, logits, img_train, label_train, BATCH_SIZE)
    print('Train Accuracy : {:.5f}'.format(train_accuracy))
    test_accuracy, _, _ = get_accuracy(sess, logits, img_test, label_test, BATCH_SIZE)
    print('Test Accuracy : {:.5f}'.format(test_accuracy))
    print('Elapsed time of an epoch: {:.5f}'.format(time.time() - epoch_start))
    if test_accuracy > max_accuracy:
        max_accuracy = test_accuracy
        print('===========Best accuracy=============')
        print(time.asctime())
        print('Best Accuracy : {:.5f}'.format(max_accuracy))
        print('Elapsed time to get the best accuracy: {:.5f}'.format(time.time() - total_start))
        if IS_SAVE:
            print('===========Saving network with the best accuracy===========')
            if not os.path.exists(CHECK_POINT_DIR):
                os.makedirs(CHECK_POINT_DIR)
            saver.save(sess, CHECK_POINT_DIR + "/model", global_step=epoch+1)

print('=====================================')

print('Final Test Accuracy : {:.5f}'.format(max_accuracy))
print('Total elapsed time: {:.5f}'.format(time.time() - total_start))

Mon May 21 20:08:52 2018
Batch size : 128, Epochs : 5, Save_dir : ./save_model
Learning Started.
Epoch : 01 , Cost : 0.23487
Train Accuracy : 0.97925
Test Accuracy : 0.97920
Elapsed time of an epoch: 14.73200
===========Best accuracy=============
Mon May 21 20:09:07 2018
Best Accuracy : 0.97920
Elapsed time to get the best accuracy: 14.73300
Epoch : 02 , Cost : 0.06431
Train Accuracy : 0.98847
Test Accuracy : 0.98720
Elapsed time of an epoch: 13.77064
===========Best accuracy=============
Mon May 21 20:09:21 2018
Best Accuracy : 0.98720
Elapsed time to get the best accuracy: 28.50364
Epoch : 03 , Cost : 0.04449
Train Accuracy : 0.99327
Test Accuracy : 0.98940
Elapsed time of an epoch: 13.39765
===========Best accuracy=============
Mon May 21 20:09:34 2018
Best Accuracy : 0.98940
Elapsed time to get the best accuracy: 41.90130
Epoch : 04 , Cost : 0.03541
Train Accuracy : 0.99255
Test Accuracy : 0.98940
Elapsed time of an epoch: 13.86289
Epoch : 05 , Cost : 0.03100
Train Accuracy : 0.994